# 第9章: ベクトル空間法 (I)

enwiki-20150112-400-r10-105752.txt.bz2は，2015年1月12日時点の英語のWikipedia記事のうち，約400語以上で構成される記事の中から，ランダムに1/10サンプリングした105,752記事のテキストをbzip2形式で圧縮したものである．このテキストをコーパスとして，単語の意味を表すベクトル（分散表現）を学習したい．第9章の前半では，コーパスから作成した単語文脈共起行列に主成分分析を適用し，単語ベクトルを学習する過程を，いくつかの処理に分けて実装する．第9章の後半では，学習で得られた単語ベクトル（300次元）を用い，単語の類似度計算やアナロジー（類推）を行う．

なお，問題83を素直に実装すると，大量（約7GB）の主記憶が必要になる． メモリが不足する場合は，処理を工夫するか，1/100サンプリングのコーパスenwiki-20150112-400-r100-10576.txt.bz2を用いよ．

In [1]:
import numpy as np

# 80. コーパスの整形
文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

+ トークンの先頭と末尾に出現する次の文字を削除: .,!?;:()[]'"
+ 空文字列となったトークンは削除

以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．

In [2]:
with open('enwiki-20150112-400-r100-10576.txt', 'r') as f:
    delete_words = ".,!?;:()[]'" + '"'
    corpus = []
    for line in f.readlines():
        for token in line[:-1].split(" "):
            if len(token) == 0:
                break
            if token[0] in delete_words:
                token = token[1:]
            if len(token) == 0:
                break
            if token[-1] in delete_words:
                token = token[:-1]
            if token:
                corpus.append(token)
with open('corpus.txt', 'w') as f:
    f.write(" ".join(corpus))

# 81. 複合語からなる国名への対処
英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．

インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．

In [3]:
# http://www.projectvisa.com/fullcountrylist.asp
with open('country.txt', 'r') as f:
    countries = [x[:-1] for x in f.readlines() if " " in x]

with open('corpus.txt', 'r') as f:
    corpus = f.read()[:-1]

In [5]:
for c in countries:
    corpus = corpus.replace(c, c.replace(" ", "_"))
with open('81.txt', 'w') as f:
    f.write(corpus)

In [14]:
corpus_list = corpus.split(" ")
dictionary = set(corpus_list)
with open('dictionary.txt', 'w') as f:
    f.write("\n".join(dictionary))

In [21]:
with open('dictionary.txt', 'r') as f:
    dictionary = [x[:-1] for x in f.readlines()]

In [23]:
corpus_index = np.array([dictionary.index(x) for x in corpus_list])

KeyboardInterrupt: 

In [24]:
corpus_index[:10]

NameError: name 'corpus_index' is not defined

# 82. 文脈の抽出
81で作成したコーパス中に出現するすべての単語$t$に関して，単語$t$と文脈語$c$のペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．

+ ある単語$t$の前後$d$単語を文脈語$c$として抽出する（ただし，文脈語に単語$t$そのものは含まない）
+ 単語$t$を選ぶ度に，文脈幅$d$は$\{1,2,3,4,5\}$の範囲でランダムに決める．

In [6]:
corpus_list = corpus.split(" ")
T = len(corpus_list)
with open("82.txt", "w") as f:
    for tn in range(T):
        d = np.random.randint(5) + 1
        t = corpus_list[tn]
        if tn - d >= 0:
            c = corpus_list[tn - d]
            if c != t:
                line = t + "\t" + c + "\n"
                f.write(line)
        if tn + d < T:
            c = corpus_list[tn + d]
            if c != t:
                line = t + "\t" + c + "\n"
                f.write(line)

# 83. 単語／文脈の頻度の計測
82の出力を利用し，以下の出現分布，および定数を求めよ．

+ $f(t,c)$: 単語$t$と文脈語$c$の共起回数
+ $f(t,∗)$: 単語$t$の出現回数
+ $f(∗,c)$: 文脈語$c$の出現回数
+ $N$: 単語と文脈語のペアの総出現回数

In [7]:
import pandas as pd
tc = pd.read_csv("82.txt", sep="\t", header=None, names=["t", "c"])

In [8]:
N = len(tc)
N

22546050

In [9]:
ft = tc.groupby('t').count()
ft.columns = [["count"]]

In [10]:
fc = tc.groupby('c').count()
fc.columns = [["count"]]

In [11]:
tc['count'] = tc['t'] 
ftc = tc.groupby(['t', 'c']).count()
ftc.columns = [["count"]]

In [12]:
ftc.head()

count
t        c                    
\t1880\n that                1
\t1952\n Is                  1
\t1991\n Fortysomething      1
\t2004\n event               1
         music               1

In [13]:
ftc.ix[('a', 'that')]

count    3345
Name: (a, that), dtype: int64

# 84. 単語文脈行列の作成
83の出力を利用し，単語文脈行列XXを作成せよ．ただし，行列XXの各要素XtcXtcは次のように定義する．

+ $f(t,c)≥10$ならば，$Xtc=PPMI(t,c)=max\{log \frac{N×f(t,c)}{f(t,∗)×f(∗,c)},0\}$
+ $f(t,c)<10$ならば，$Xtc=0$

ここで，$PPMI(t,c)$はPositive Pointwise Mutual Information（正の相互情報量）と呼ばれる統計量である．なお，行列XXの行数・列数は数百万オーダとなり，行列のすべての要素を主記憶上に載せることは無理なので注意すること．幸い，行列XXのほとんどの要素は00になるので，非00の要素だけを書き出せばよい．

# 85. 主成分分析による次元圧縮
84で得られた単語文脈行列に対して，主成分分析を適用し，単語の意味ベクトルを300次元に圧縮せよ．

# 86. 単語ベクトルの表示
85で得た単語の意味ベクトルを読み込み，"United States"のベクトルを表示せよ．ただし，"United States"は内部的には"United_States"と表現されていることに注意せよ．

# 87. 単語の類似度
85で得た単語の意味ベクトルを読み込み，"United States"と"U.S."のコサイン類似度を計算せよ．ただし，"U.S."は内部的に"U.S"と表現されていることに注意せよ．

# 88. 類似度の高い単語10件
85で得た単語の意味ベクトルを読み込み，"England"とコサイン類似度が高い10語と，その類似度を出力せよ．

# 89. 加法構成性によるアナロジー
85で得た単語の意味ベクトルを読み込み，vec("Spain") - vec("Madrid") + vec("Athens")を計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ